# 5 - Footprint Geometry Check (OrbitCheck)

This notebook checks if the claimed/observed image footprint is physically plausible
given the satellite altitude and sensor Field-of-View (FOV).

Outputs:
- Footprint feasibility report saved to: `data/outputs/reports/`


In [ ]:
import os, json, math

PROJECT_ROOT = "/content/orbitcheck"

META_DIR = os.path.join(PROJECT_ROOT, "data/raw/metadata")
REPORT_DIR = os.path.join(PROJECT_ROOT, "data/outputs/reports")

os.makedirs(REPORT_DIR, exist_ok=True)

print("META_DIR:", META_DIR)
print("REPORT_DIR:", REPORT_DIR)


In [ ]:
meta_files = [f for f in os.listdir(META_DIR) if f.endswith(".json")]
if len(meta_files) == 0:
    raise ValueError("No metadata JSON found. Run Notebook 01 first.")

print("Available metadata files:")
for f in meta_files:
    print(" -", f)

meta_filename = input("\nEnter metadata filename (example: sample1.json): ").strip()
meta_path = os.path.join(META_DIR, meta_filename)

with open(meta_path, "r") as f:
    meta = json.load(f)

print("\n Loaded metadata:")
print(json.dumps(meta, indent=2))


In [ ]:
image_id = meta["image_id"]

orbit_report_path = os.path.join(REPORT_DIR, f"{image_id}_orbit_report.json")

if not os.path.exists(orbit_report_path):
    print("Orbit report not found. Run Notebook 02 first OR enter altitude manually.")
    altitude_km = float(input("Enter approximate altitude (km): "))
else:
    with open(orbit_report_path, "r") as f:
        orbit_report = json.load(f)
    altitude_km = float(orbit_report["approx_altitude_km"])
    print("Loaded altitude from orbit report:", altitude_km, "km")


In [ ]:
image_id = meta["image_id"]

orbit_report_path = os.path.join(REPORT_DIR, f"{image_id}_orbit_report.json")

if not os.path.exists(orbit_report_path):
    print("Orbit report not found. Run Notebook 02 first OR enter altitude manually.")
    altitude_km = float(input("Enter approximate altitude (km): "))
else:
    with open(orbit_report_path, "r") as f:
        orbit_report = json.load(f)
    altitude_km = float(orbit_report["approx_altitude_km"])
    print("Loaded altitude from orbit report:", altitude_km, "km")


In [ ]:
print("\nChoose footprint input mode:")
print("1) Manual footprint (km)")
print("2) Use metadata footprint (if present)")

mode = input("Enter 1 or 2: ").strip()

if mode == "1":
    footprint_width_km = float(input("Enter approximate footprint width (km): "))
    footprint_height_km = float(input("Enter approximate footprint height (km): "))
    footprint_source = "manual"

elif mode == "2":
    fp = meta.get("footprint", None)

    if fp is None:
        print("No footprint in metadata. Switching to manual.")
        footprint_width_km = float(input("Enter approximate footprint width (km): "))
        footprint_height_km = float(input("Enter approximate footprint height (km): "))
        footprint_source = "manual"
    else:
        # if your footprint is stored as bbox: {"width_km":..., "height_km":...}
        footprint_width_km = float(fp.get("width_km"))
        footprint_height_km = float(fp.get("height_km"))
        footprint_source = "metadata"
else:
    raise ValueError("Invalid choice")

print("\n Footprint used:", footprint_width_km, "km x", footprint_height_km, "km")


In [ ]:
print("\nEnter sensor Field-of-View (FOV) assumptions:")
fov_deg = float(input("Enter approximate sensor FOV in degrees (example: 20): "))

fov_rad = math.radians(fov_deg)
expected_width_km = 2 * altitude_km * math.tan(fov_rad / 2)

print("\n Expected footprint width (km) from altitude + FOV:", round(expected_width_km, 2))


In [ ]:
error_pct = abs(footprint_width_km - expected_width_km) / expected_width_km * 100

if error_pct < 20:
    verdict = "FEASIBLE"
elif error_pct < 50:
    verdict = "UNCERTAIN / SUSPICIOUS"
else:
    verdict = "INFEASIBLE"

print("\n====================")
print("Footprint width (observed):", footprint_width_km, "km")
print("Footprint width (expected):", round(expected_width_km, 2), "km")
print("Error %:", round(error_pct, 2))
print("Verdict:", verdict)
print("====================")


In [ ]:
footprint_report = {
    "image_id": image_id,
    "altitude_km": altitude_km,
    "footprint_input": {
        "width_km": footprint_width_km,
        "height_km": footprint_height_km,
        "source": footprint_source
    },
    "sensor_assumption": {
        "fov_deg": fov_deg
    },
    "expected_footprint_width_km": expected_width_km,
    "error_pct": error_pct,
    "verdict": verdict,
    "notes": "MVP geometry check. Later extend using real satellite sensor profiles + georeferenced footprints."
}

out_path = os.path.join(REPORT_DIR, f"{image_id}_footprint_report.json")
with open(out_path, "w") as f:
    json.dump(footprint_report, f, indent=2)

print("\nFootprint report saved:", out_path)
print(json.dumps(footprint_report, indent=2))
